<a href="https://colab.research.google.com/github/jwang44/Try-colabing-in-colab/blob/main/credit_card_new_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECSE 551 Mini-project 1
*Group 10: Junhao Wang, Yinan Zhou, and Ruilin Ji*

This notebook is dedicated for the credit card dataset, including the model, cross validation, and various experiment. 

## Start here

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/My Drive/"

/content/drive/My Drive


In [3]:
import numpy as np
import pandas as pd
import scipy.stats
import statistics

## Credit Card Dataset

In [4]:
# generate new feature by multiplication and normalize
def newfeature(x,y):
  z=x*y
  norz=scipy.stats.zscore(z, axis=0, ddof=0, nan_policy='propagate')
  return norz

In [6]:
# convert csv to dataframe
df = pd.read_csv('creditcard.csv')
original_data = df.to_numpy()

In [7]:
# normalize feature
NorData = scipy.stats.zscore(df.iloc[:,:-1], axis=0, ddof=0, nan_policy='propagate')
# normalized dataset
NorDataset = np.column_stack((NorData,df.iloc[:,-1]))

New feature

In [8]:
NewF1 = newfeature(df.V3, df.V7)
NewF2 = newfeature(df.V11, df.V12)
NewF3 = newfeature(df.V12, df.V16)
NewF4 = newfeature(df.V16, df.V17)
NewF5 = newfeature(df.V16, df.V18)
NewF6 = newfeature(df.V17, df.V18)
# new feature
NewF = np.column_stack((NewF1,NewF2,NewF3,NewF4,NewF5,NewF6))

In [9]:
# normalized dataset with new feature
NorDatasetNew = np.column_stack((NorData,NewF,df.iloc[:,-1]))

## The model

In [13]:
# sigmoid function
def sigmoid(a):
  return 1/(1+np.exp(-a))

class Logistic_regression():
  def __init__(self):#,X_train,y_train,learning_rate,X_test,y_test):
    pass
    
  # training
  def fit(self,X_train,y_train,learning_rate):
    #n,m = np.shape(self.X_train)
    n,m = np.shape(X_train)  
    wk = np.ones([m+1,1]) # wk weights, initialized with 1
    wk1 = np.zeros([m+1,1])# wk+1 weights,initialized with 0          
    itrnum = 500       # max number of iterations 
    e = 0.001
    der = 0
    for k in range(0,itrnum):
      for i in range(0,n):
        #xi = self.X_train[i].T
        xi = X_train[i].T
        x0 = np.array([1])
        xi = np.concatenate((xi, x0),axis = 0)
        #yi = self.y_train[i]
        yi = y_train[i]
        der = der-xi*(yi-sigmoid(np.matmul(wk[:,0].T,xi))) # take derivative w.r.t w
      #wk1[:,0] = wk[:,0]-self.learning_rate*der       # update rule
      wk1[:,0] = wk[:,0]-learning_rate*der       # update rule
      if (np.linalg.norm(wk1[:,0]-wk[:,0]))**2<e:         
        break 
      else:
        wk = wk1.copy()
    return wk1
  
  # validation
  def predict(self,w,X_test):
    #n,m = np.shape(self.X_test)
    n,m = np.shape(X_test)   
    y_predict = np.zeros([n,1])
    for i in range(0,n):
      #xi = self.X_test[i].T
      xi = X_test[i].T
      x0 = np.array([1])
      xi = np.concatenate((xi, x0),axis = 0)
      p1 = sigmoid(np.matmul(w.T,xi)) # calculate probabilities p(y=1|x)
      # covert probabilities to 0 or 1 by thresholding at 0.5
      if p1>=0.5:
        y_predict[i] = 1
      else:
        y_predict[i] = 0
    return y_predict

  # evaluate accuracy
  def Accu_eval(self,y_test,y_predict):
    #y_predict = self.predict(X_test)
    n,j = np.shape(y_predict)
    TP = 0;FP = 0;TN = 0;FN = 0
    # count TP,TN,FP,FN in validation set
    '''for i in range(n):
      if  self.y_test[i]==1 and y_predict[i]==1:
        TP = TP+1
      elif self.y_test[i]==0 and y_predict[i]==0:
        TN = TN+1
      elif self.y_test[i]==0 and y_predict[i]==1:
        FP = FP+1
      elif self.y_test[i]==1 and y_predict[i]==0:
        FN = FN+1'''
    for i in range(n):
      if  y_test[i]==1 and y_predict[i]==1:
        TP = TP+1
      elif y_test[i]==0 and y_predict[i]==0:
        TN = TN+1
      elif y_test[i]==0 and y_predict[i]==1:
        FP = FP+1
      elif y_test[i]==1 and y_predict[i]==0:
        FN = FN+1    
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    F = 2*precision*recall/(precision+recall)
    specificity = TN/(FP+TN)
    FPR = FP/(FP+TN)
    print("accuracy:",accuracy)
    # print("precision:",precision)
    # print("recall:",recall)
    # print("F:",F)
    # print("specificity:",specificity)
    # print("False Positive Rate:",FPR)
    print("")
    return accuracy
    

## Cross validation

In [14]:
class Cross_validation():
  def __init__(self, k):
    # k: k-fold
    self.k = k

  def prepare_data(self, data):
    # data: np array converted from csv
    np.random.shuffle(data)
    X = data[:, :-1]  # features
    y = data[:, -1]   # labels

    # split data into k equal segments, assign them to train and test later
    Xs = np.array_split(X, self.k, axis=0)
    ys = np.array_split(y, self.k, axis=0)
    return Xs, ys

  def get_accuracy(self, Xs, ys, lr):
    accu_trains = []
    accu_tests = []
    for i in range(self.k):
      X_cv = Xs[:] # X_cross_validation
      y_cv = ys[:] # y_cross_validation

      X_test = X_cv.pop(i)
      y_test = y_cv.pop(i)

      X_train = np.concatenate(X_cv)
      y_train = np.concatenate(y_cv)

      model = Logistic_regression()
      w = model.fit(X_train, y_train, lr)

      print("----------FOLD ", i+1, "----------")

      print("----Train----")
      y_predict_train = model.predict(w, X_train)
      accu_train = model.Accu_eval(y_train, y_predict_train)
      accu_trains.append(accu_train)

      print("----Validation----")
      y_predict_test = model.predict(w, X_test)
      accu_test = model.Accu_eval(y_test, y_predict_test)
      accu_tests.append(accu_test)

    return np.mean(accu_trains), np.mean(accu_tests)


## Experiment with different learning rates

In [15]:
lrs = np.logspace(-5, -1, 5) # different learning rates to try
# or we can also try other hyperparameters here
cv = Cross_validation(5) # 5-fold cross-validation
Xs, ys = cv.prepare_data(original_data)
for lr in lrs:
  print("---------------LEARNING RATE: ", lr, "---------------")
  accu_train_avg, accu_val_avg = cv.get_accuracy(Xs, ys, lr)
  print("---------------TRAIN AVERAGE ACCURACY", accu_train_avg, "---------------")
  print("---------------VALIDATION AVERAGE ACCURACY", accu_val_avg, "---------------")
  print("\n-------------------------------------------------------------------------------\n")


---------------LEARNING RATE:  1e-05 ---------------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


----------FOLD  1 ----------
----Train----
accuracy: 0.8737373737373737

----Validation----
accuracy: 0.914572864321608

----------FOLD  2 ----------
----Train----
accuracy: 0.9407313997477932

----Validation----
accuracy: 0.9444444444444444

----------FOLD  3 ----------
----Train----
accuracy: 0.9508196721311475

----Validation----
accuracy: 0.9292929292929293

----------FOLD  4 ----------
----Train----
accuracy: 0.9445145018915511

----Validation----
accuracy: 0.9393939393939394

----------FOLD  5 ----------
----Train----
accuracy: 0.935687263556116

----Validation----
accuracy: 0.9646464646464646

---------------TRAIN AVERAGE ACCURACY 0.9290980422127963 ---------------
---------------VALIDATION AVERAGE ACCURACY 0.9384701284198771 ---------------

-------------------------------------------------------------------------------

---------------LEARNING RATE:  0.0001 ---------------
----------FOLD  1 ----------
----Train----
accuracy: 0.9583333333333334

----Validation----
accuracy: 0.9

## Experiment with different features

During the experiment on different learning rates, we found that the best learning rate is **0.0001**, so we use this learning rate for our experiment with different feature selections. 

In [16]:
lr = 0.0001
cv = Cross_validation(5) # 5-fold cross-validation
Xs, ys = cv.prepare_data(NorDataset)
print("----------Using normalized features, without new features----------")
accu_avg_train, accu_avg_val = cv.get_accuracy(Xs, ys, lr)
print("----------AVERAGE ACCURACY", accu_avg_val, "----------")
print("\n---------------------------------------------------------------------")


----------Using normalized features, without new features----------
----------FOLD  1 ----------
----Train----
accuracy: 0.952020202020202

----Validation----
accuracy: 0.964824120603015

----------FOLD  2 ----------
----Train----
accuracy: 0.9382093316519546

----Validation----
accuracy: 0.9494949494949495

----------FOLD  3 ----------
----Train----
accuracy: 0.9546027742749055

----Validation----
accuracy: 0.9646464646464646

----------FOLD  4 ----------
----Train----
accuracy: 0.9445145018915511

----Validation----
accuracy: 0.9343434343434344

----------FOLD  5 ----------
----Train----
accuracy: 0.9520807061790668

----Validation----
accuracy: 0.9292929292929293

----------AVERAGE ACCURACY (0.948285503203536, 0.9485203796761585) ----------

---------------------------------------------------------------------


In [17]:
lr = 0.0001
cv = Cross_validation(5) # 5-fold cross-validation
Xs, ys = cv.prepare_data(NorDatasetNew)
print("----------Using normalized features, with new features----------")
accu_avg_train, accu_avg_val = cv.get_accuracy(Xs, ys, lr)
print("----------AVERAGE ACCURACY", accu_avg_val, "----------")
print("\n---------------------------------------------------------------------")


----------Using normalized features, with new features----------
----------FOLD  1 ----------
----Train----
accuracy: 0.9583333333333334

----Validation----
accuracy: 0.964824120603015

----------FOLD  2 ----------
----Train----
accuracy: 0.9558638083228247

----Validation----
accuracy: 0.9444444444444444

----------FOLD  3 ----------
----Train----
accuracy: 0.9583858764186634

----Validation----
accuracy: 0.9343434343434344

----------FOLD  4 ----------
----Train----
accuracy: 0.9558638083228247

----Validation----
accuracy: 0.9393939393939394

----------FOLD  5 ----------
----Train----
accuracy: 0.9609079445145019

----Validation----
accuracy: 0.9545454545454546

----------AVERAGE ACCURACY 0.9475102786660574 ----------

---------------------------------------------------------------------
